In [54]:
import numpy as np 
import pandas as pd

In [282]:
train_data = pd.read_csv('D:\семинары по python\Датасеты\Titanic_train.csv',index_col='PassengerId')
test_data = pd.read_csv('D:\семинары по python\Датасеты\Titanic_test.csv',index_col='PassengerId')

In [283]:
train_data = pd.concat([train_data, test_data])

In [284]:
num_cols = [
    'Pclass',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
]

cat_cols = [
    'Sex',
    'Embarked',
]

target_col = 'Survived'

cols = num_cols + cat_cols + [target_col]
data = train_data[cols]

In [285]:
columns = ('Pclass Age SibSp Parch Fare Sex Embarked Survived')
numeric_indices = np.array([0, 1, 2, 3, 4, 7])
categorical_indices = np.array([5, 6])
data.columns = columns.split() 
#data = data.dropna()


# Посмотрим на распределение категориальных фичей
for col in cat_cols:
    print(f"{col} DISTRIBUTION")
    print(data[col].value_counts())
    print()
# способ убрать колонки 
#data = data.drop(columns=['currency'])
#cat_cols.remove('currency')

In [286]:
numeric_data = data[data.columns[numeric_indices]]
categorial_data = data[data.columns[categorical_indices]]


In [287]:
dummy_features = pd.get_dummies(categorial_data)
data = pd.concat([numeric_data, dummy_features], axis=1)
X_origin = data.iloc[:, :-1]


In [288]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = data.iloc[0:891, :11]
X_test = data.iloc[891:1310, :11]
Y_train = X_train['Survived']
X_train.drop('Survived', axis=1, inplace=True)
X_test.drop('Survived', axis=1, inplace=True)



In [289]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)

In [290]:
#!pip install xgboost


grid = {'max_depth':10}
 
clf = XGBClassifier()
clf.max_depth
3
clf.set_params(**grid)
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
clf.max_depth
10

In [291]:
import xgboost

In [292]:
grid = {'max_depth':10,
        'n_estimators':100,
         'objective':'binary:logistic',
         'booster' :'gbtree'}
boosting_model = xgboost.XGBClassifier(use_label_encoder=False)
boosting_model.set_params(**grid)
boosting_model.fit(X_train, Y_train)

y_train_predicted = boosting_model.predict_proba(X_train)[:, 1]
y_test_predicted = boosting_model.predict_proba(X_test)[:, 1]


[15:20:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [293]:
y_test_predicted=y_test_predicted.round(0)
y_train_predicted=y_train_predicted.round(0)

In [295]:
from sklearn.metrics import roc_auc_score, roc_curve

In [296]:
train_auc = roc_auc_score(Y_train, y_train_predicted)

In [297]:
train_auc

0.9775429009682677

In [298]:
X_test_copy=X_test.copy()
X_test_copy=X_test_copy.drop(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],axis=1)
X_test_copy['Survived']=y_test_predicted

In [299]:
#df = df.astype({'a': np.float})
X_test_int=X_test_copy.copy()
X_test_int=X_test_int.astype({'Survived': np.int64})

In [300]:
X_test_int.head(20)

,Survived
PassengerId,
892,0
893,0
894,1
895,1
896,1
897,0
898,0
899,0
900,1


In [301]:
X_test_int.to_csv('my_submission.csv')